In [2]:
import dash
import datetime
import calendar
import plotly.express as px
import pandas as pd
from  dash import Dash, html, dcc
from  dash.dependencies import Output, Input  
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt, timedelta
import plotly.graph_objects as go

df = pd.read_csv(r"dataset.csv")
df['Processed_Month'] = pd.to_datetime(df['Processed_Month'])
df_long = pd.DataFrame(df)
# print(df_long)
df_long.to_csv("df_long.csv")
# Aggregate the data by counting occurrences 
#Converting to wide-format for plotly express
df_aggregated = df_long.groupby(['Processed_Month', 'Ratings']).size().unstack(fill_value=0)
df_aggregated.columns.name = None 
df_aggregated = df_aggregated.rename(columns={1.0: 'Rating_1', 2.0: 'Rating_2', 3.0: 'Rating_3', 4.0: 'Rating_4', 5.0: 'Rating_5'})

df_aggregated.reset_index(inplace=True)
df_aggregated['Total_Votes']=df_aggregated[['Rating_1','Rating_2',"Rating_3",'Rating_4','Rating_5']].sum(axis=1)
df_aggregated['Average_Rating']=None
# df_aggregated["Color"]=None
df_aggregated["Popularity_Index"]=None
rating_weight = 0.0
for index, row in df_aggregated.iterrows():
    rating_weight = 0.0
    for rating in (['Rating_1','Rating_2',  'Rating_3', 'Rating_4',  'Rating_5']):
        rating_weight += float(rating[7:])*row[rating]
    # df_aggregated.at[index,'Color'] = 1
    if row["Total_Votes"]!=0:
        df_aggregated.at[index,'Average_Rating']=(rating_weight/row["Total_Votes"])
    else:
        row["Average_Rating"]=0
    popularity_numerator = 1
    popularity_denominator = 1
    if row['Rating_1'] != 0 and row['Rating_2'] != 0:
        popularity_denominator *= (row['Rating_1'] * row['Rating_2'])
    elif row['Rating_1'] == 0 and row['Rating_2'] != 0:
        popularity_denominator *= row['Rating_2']
    elif row['Rating_1'] != 0 and row['Rating_2'] == 0:
        popularity_denominator *= row['Rating_1']
    else:
        popularity_denominator=1
    if row['Rating_4'] != 0 and row['Rating_5'] != 0:
        popularity_numerator *= (row['Rating_4'] * row['Rating_5'])
    elif row['Rating_4'] == 0 and row['Rating_5'] != 0:
        popularity_denominator *= row['Rating_5']
    elif row['Rating_4'] != 0 and row['Rating_5'] == 0:
        popularity_denominator *= row['Rating_4']
    # print(f'denominator = {popularity_denominator} numerator = {popularity_numerator}')
    # print(f'Rating-1 ={row["Rating_1"]},Rating-2 ={row["Rating_2"]},Rating-4 ={row["Rating_4"]},Rating-5 ={row["Rating_5"]}')
    df_aggregated.at[index, 'Popularity_Index'] = round(np.log((popularity_numerator) / (popularity_denominator)), 2)
print(df_aggregated)

df_aggregated.to_csv('bin1.csv')
df_bin2 = df_aggregated
df_bin3 = df_aggregated
df_bin4 = df_aggregated

#Transforming based on bin size 2************************************************************************************
df_bin2['1'] = df_bin2['Rating_1'].shift(1)
df_bin2['2'] = df_bin2['Rating_2'].shift(1)
df_bin2['3'] = df_bin2['Rating_3'].shift(1)
df_bin2['4'] = df_bin2['Rating_4'].shift(1)
df_bin2['5'] = df_bin2['Rating_5'].shift(1)
df_bin2['6'] = df_bin2['Total_Votes'].shift(1)


df_bin2.fillna(0, inplace = True)


df_bin2['Rating_1'] = df_bin2['Rating_1']+ df_bin2['1']
df_bin2['Rating_2'] = df_bin2['Rating_2']+ df_bin2['2']
df_bin2['Rating_3'] = df_bin2['Rating_3']+ df_bin2['3']
df_bin2['Rating_4'] = df_bin2['Rating_4']+ df_bin2['4']
df_bin2['Rating_5'] = df_bin2['Rating_5']+ df_bin2['5']
df_bin2['Total_Votes'] = df_bin2['Total_Votes']+ df_bin2['6']

# print(df_bin2)
df_bin2 = df_bin2.iloc[::-1][::2]

df_bin2 = df_bin2.iloc[::-1]
df_bin2.reset_index(drop=True, inplace=True)


#Add average ratings to the df
for index, row in df_bin2.iterrows():
    rating_weight = 0.0
    for rating in (['Rating_1','Rating_2',  'Rating_3', 'Rating_4',  'Rating_5']):
        rating_weight += float(rating[7:])*row[rating]
    df_bin2.at[index,'Color'] = 1
    if row['Total_Votes'] != 0:
        df_bin3.at[index, 'Average Rating'] = round(rating_weight / row['Total_Votes'], 2)
    else:
        df_bin3.at[index, 'Average Rating'] = 0.00

for index, row in df_bin2.iterrows():
    popularity_numerator = 1
    popularity_denominator = 1
    if row['Rating_1'] != 0 and row['Rating_2'] != 0:
        popularity_denominator *= row['Rating_1'] * row['Rating_2']
    elif row['Rating_1'] == 0 and row['Rating_2'] != 0:
        popularity_denominator *= row['Rating_2']
    elif row['Rating_1'] != 0 and row['Rating_2'] == 0:
        popularity_denominator *= row['Rating_1']
    else:
        popularity_denominator = 1
    
    if row['Rating_4'] != 0 and row['Rating_5'] != 0:
        popularity_numerator *= row['Rating_4'] * row['Rating_5']
    elif row['Rating_4'] == 0 and row['Rating_5'] != 0:
        popularity_numerator *= row['Rating_5']
    elif row['Rating_4'] != 0 and row['Rating_5'] == 0:
        popularity_numerator *= row['Rating_4']
    else:
        popularity_numerator = 1
    
    # Update the 'Popularity_Index' column in the original DataFrame
    df_bin2.at[index, 'Popularity_Index'] = round(np.log((popularity_numerator) / (popularity_denominator)), 2)

df_bin22 = df_bin2
df_bin22['Rating_1'] = df_bin22['1']
df_bin22['Rating_2'] = df_bin22['2']
df_bin22['Rating_3'] = df_bin22['3']
df_bin22['Rating_4'] = df_bin22['4']
df_bin22['Rating_5'] = df_bin22['5']

df_bin22.to_csv(r"final-ouput-3(binsize).csv", index=False)
# print(df_bin22)

# print("Printing bin 1")
# print(df_aggregated['Total_Votes'].sum())
# print("Printing bin2")

# print(df_bin2['Total_Votes'].sum())

# Transforming based on bin size 3
df_bin3['1'] = df_bin3['Rating_1'].shift(3)
df_bin3['2'] = df_bin3['Rating_2'].shift(3)
df_bin3['3'] = df_bin3['Rating_3'].shift(3)
df_bin3['4'] = df_bin3['Rating_4'].shift(3)
df_bin3['5'] = df_bin3['Rating_5'].shift(3)
df_bin3['6'] = df_bin3['Total_Votes'].shift(3)

df_bin3.fillna(0, inplace=True)

df_bin3['Rating_1'] = df_bin3['Rating_1'] + df_bin3['1']
df_bin3['Rating_2'] = df_bin3['Rating_2'] + df_bin3['2']
df_bin3['Rating_3'] = df_bin3['Rating_3'] + df_bin3['3']
df_bin3['Rating_4'] = df_bin3['Rating_4'] + df_bin3['4']
df_bin3['Rating_5'] = df_bin3['Rating_5'] + df_bin3['5']
df_bin3['Total_Votes'] = df_bin3['Total_Votes'] + df_bin3['6']

df_bin3 = df_bin3.iloc[::-1][::3]
df_bin3 = df_bin3.iloc[::-1]
df_bin3.reset_index(drop=True, inplace=True)

for index, row in df_bin3.iterrows():
    rating_weight = 0.0
    for rating in (['Rating_1', 'Rating_2', 'Rating_3', 'Rating_4', 'Rating_5']):
        rating_weight += float(rating[7:]) * row[rating]
    df_bin3.at[index, 'Color'] = 1
    if row['Total_Votes'] != 0:
        df_bin3.at[index, 'Average Rating'] = round(rating_weight / row['Total_Votes'], 2)
    else:
        df_bin3.at[index, 'Average Rating'] = 0.00

for index, row in df_bin3.iterrows():
    popularity_numerator = 1
    popularity_denominator = 1
    if row['Rating_1'] != 0 and row['Rating_2'] != 0:
        popularity_denominator *= row['Rating_1'] * row['Rating_2']
    elif row['Rating_1'] == 0 and row['Rating_2'] != 0:
        popularity_denominator *= row['Rating_2']
    elif row['Rating_1'] != 0 and row['Rating_2'] == 0:
        popularity_denominator *= row['Rating_1']
    else:
        popularity_denominator = 1
    
    if row['Rating_4'] != 0 and row['Rating_5'] != 0:
        popularity_numerator *= row['Rating_4'] * row['Rating_5']
    elif row['Rating_4'] == 0 and row['Rating_5'] != 0:
        popularity_numerator *= row['Rating_5']
    elif row['Rating_4'] != 0 and row['Rating_5'] == 0:
        popularity_numerator *= row['Rating_4']
    else:
        popularity_numerator = 1
    
    # Update the 'Popularity_Index' column in the original DataFrame
    df_bin3.at[index, 'Popularity_Index'] = round(np.log((popularity_numerator) / (popularity_denominator)), 2)


df_bin33 = df_bin3.copy()
df_bin33['Rating_1'] = df_bin33['1']
df_bin33['Rating_2'] = df_bin33['2']
df_bin33['Rating_3'] = df_bin33['3']
df_bin33['Rating_4'] = df_bin33['4']
df_bin33['Rating_5'] = df_bin33['5']

df_bin33.to_csv(r"bin3.csv", index=False)

# Transforming based on bin size 4
df_bin4['1'] = df_bin4['Rating_1'].shift(4)
df_bin4['2'] = df_bin4['Rating_2'].shift(4)
df_bin4['3'] = df_bin4['Rating_3'].shift(4)
df_bin4['4'] = df_bin4['Rating_4'].shift(4)
df_bin4['5'] = df_bin4['Rating_5'].shift(4)
df_bin4['6'] = df_bin4['Total_Votes'].shift(4)

df_bin4.fillna(0, inplace=True)

df_bin4['Rating_1'] = df_bin4['Rating_1'] + df_bin4['1']
df_bin4['Rating_2'] = df_bin4['Rating_2'] + df_bin4['2']
df_bin4['Rating_3'] = df_bin4['Rating_3'] + df_bin4['3']
df_bin4['Rating_4'] = df_bin4['Rating_4'] + df_bin4['4']
df_bin4['Rating_5'] = df_bin4['Rating_5'] + df_bin4['5']
df_bin4['Total_Votes'] = df_bin4['Total_Votes'] + df_bin4['6']

df_bin4 = df_bin4.iloc[::-1][::4]
df_bin4 = df_bin4.iloc[::-1]
df_bin4.reset_index(drop=True, inplace=True)

for index, row in df_bin4.iterrows():
    rating_weight = 0.0
    for rating in (['Rating_1', 'Rating_2', 'Rating_3', 'Rating_4', 'Rating_5']):
        rating_weight += float(rating[7:]) * row[rating]
    df_bin4.at[index, 'Color'] = 1
    if row['Total_Votes'] != 0:
        df_bin4.at[index, 'Average Rating'] = round(rating_weight / row['Total_Votes'], 2)
    else:
        df_bin4.at[index, 'Average Rating'] = 0.00
    
for index, row in df_bin4.iterrows():
    popularity_numerator = 1
    popularity_denominator = 1
    if row['Rating_1'] != 0 and row['Rating_2'] != 0:
        popularity_denominator *= row['Rating_1'] * row['Rating_2']
    elif row['Rating_1'] == 0 and row['Rating_2'] != 0:
        popularity_denominator *= row['Rating_2']
    elif row['Rating_1'] != 0 and row['Rating_2'] == 0:
        popularity_denominator *= row['Rating_1']
    else:
        popularity_denominator = 1
    
    if row['Rating_4'] != 0 and row['Rating_5'] != 0:
        popularity_numerator *= row['Rating_4'] * row['Rating_5']
    elif row['Rating_4'] == 0 and row['Rating_5'] != 0:
        popularity_numerator *= row['Rating_5']
    elif row['Rating_4'] != 0 and row['Rating_5'] == 0:
        popularity_numerator *= row['Rating_4']
    else:
        popularity_numerator = 1
    
    # Update the 'Popularity_Index' column in the original DataFrame
    df_bin4.at[index, 'Popularity_Index'] = round(np.log((popularity_numerator) / (popularity_denominator)), 2)


df_bin44 = df_bin4.copy()
df_bin44['Rating_1'] = df_bin44['1']
df_bin44['Rating_2'] = df_bin44['2']
df_bin44['Rating_3'] = df_bin44['3']
df_bin44['Rating_4'] = df_bin44['4']
df_bin44['Rating_5'] = df_bin44['5']

df_bin44.to_csv(r"bin4.csv", index=False)
df_bin22.to_csv("bin2.csv",index=False)

   Processed_Month  Rating_1  Rating_2  Rating_3  Rating_4  Rating_5  \
0       2021-03-01         8         2         3         4         5   
1       2021-04-01        50        10         8        60       167   
2       2021-05-01        42        12        14        30        49   
3       2021-06-01        12         9         6        24        26   
4       2021-07-01        35         8         6        29        36   
5       2021-08-01        58         8        21        41       104   
6       2021-09-01       147        22        53       145       424   
7       2021-10-01       283        51       173       405       899   
8       2021-11-01       118        33        67       174       412   
9       2021-12-01        78        24        38        85       252   
10      2022-01-01        79        27        52       100       334   
11      2022-02-01       100        31        44        98       323   
12      2022-03-01       103        27        35        78      

C:\Users\harsh\AppData\Local\Temp\ipykernel_2540\3661537556.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Processed_Month'] = pd.to_datetime(df['Processed_Month'])
